### Dataset

In [84]:
import pandas as pd
from auto_ts import auto_timeseries
import dill
import talib
import numpy as np
import warnings

warnings.filterwarnings('ignore')

from sklearn.model_selection import TimeSeriesSplit
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization
from keras.regularizers import l2
from keras.optimizers import Adam, RMSprop, SGD
from scikeras.wrappers import KerasRegressor
from keras.callbacks import EarlyStopping
from skopt import BayesSearchCV
from sklearn.preprocessing import LabelEncoder
import tensorflow.keras.backend as K
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from keras.losses import categorical_crossentropy
from sklearn.metrics import log_loss
from keras.utils import to_categorical
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier

In [85]:
# Evito que ciertas columnas se transformen a notacion cientifica en las predicciones
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

In [86]:
columns = [
    'Open_time',
    'Open',
    'High',
    'Low',
    # 'Close',
    'Number of trades',
    'Close_BTCUSDT',
    'Volume_BTCUSDT',
    'Number_of_trades_BTCUSDT',
    'Close_ETHUSDT',
    'Volume_ETHUSDT',
    'Number_of_trades_ETHUSDT',
    'Close_BNBUSDT',
    'Volume_BNBUSDT',
    'Number_of_trades_BNBUSDT',
    'SMA_20',
    'EMA_20',
    'Upper_Band',
    'Middle_Band',
    'Lower_Band',
    'RSI',
    'buy_1000x_high_coinbase',
    'sell_1000x_high_coinbase',
    'total_trades_coinbase',	
    'Tweets_Utilizados',
    'Tweets_Utilizados_coin',
    'Tweets_Utilizados_referentes',
    'Tweets_Utilizados_whale_alert',
    'Buy_1000x_high',
    'sell_1000x_high',
    'total_trades_binance'
]

### Armado y entrenamiento de un clasificador a partir de los datos originales

#### Modelo Light GBM

In [87]:
complete_dataset = pd.read_csv('/Users/mmarchetta/Desktop/Tesis-2024/data-visualization/final_dataset.csv') 
classifier_dataset = complete_dataset[columns]
classifier_dataset['Open_time'] = pd.to_datetime(classifier_dataset['Open_time'])
classifier_dataset['Tendencia'] = complete_dataset['Tendencia']

clasifier_validation = classifier_dataset[-5:]
classifier_dataset = classifier_dataset[:-5]

In [88]:
display(classifier_dataset.tail())

,Open_time,Open,High,Low,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,Volume_ETHUSDT,Number_of_trades_ETHUSDT,Close_BNBUSDT,Volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance,Tendencia
898,2024-03-13,10.74,11.46,10.64,276468.00,73072.41,52659.71,2501197.00,4004.79,482305.78,1536498.00,630.50,2526002.56,1265237.00,9.45,9.70,11.82,9.45,7.07,73.43,64.00,81.00,92576.00,275,205,1.00,73.00,578.00,553.00,164000.00,Alcista
899,2024-03-14,11.37,11.89,10.68,536988.00,71388.94,71757.63,2994869.00,3881.70,648237.52,1919963.00,603.20,2119540.30,1038297.00,9.65,9.88,12.03,9.65,7.27,74.51,102.00,133.00,145727.00,211,181,4.00,29.00,754.00,677.00,327000.00,Alcista
900,2024-03-15,11.56,11.71,9.97,557152.00,69499.85,103334.04,3904445.00,3742.19,947537.41,2487337.00,632.70,3066312.79,1365283.00,9.80,9.97,12.07,9.80,7.52,63.27,88.00,83.00,147460.00,238,106,0.00,25.00,493.00,430.00,360000.00,Bajista
901,2024-03-16,10.81,10.90,9.50,330505.00,65300.63,55926.95,2729019.00,3523.09,548288.16,1798939.00,576.40,1811838.04,1025452.00,9.89,9.94,11.99,9.89,7.78,50.82,30.00,49.00,88095.00,670,471,0.00,20.00,513.00,403.00,209000.00,Bajista
902,2024-03-17,9.68,10.25,9.19,229683.00,68393.48,49742.22,2449156.00,3644.71,517790.99,1721355.00,571.70,1712920.34,802297.00,9.98,9.95,11.93,9.98,8.04,54.27,36.00,48.00,71390.00,693,413,0.00,21.00,295.00,277.00,150000.00,Alcista


In [89]:
classifier_dataset.shape

(903, 31)

In [90]:
X = classifier_dataset.drop(columns=["Tendencia", "Open_time"])
y = classifier_dataset["Tendencia"]

# label_encoder = LabelEncoder()
# y_encoded = label_encoder.fit_transform(classifier_dataset["Tendencia"])

y = y.to_numpy().reshape(-1, 1)
onehot_encoder = OneHotEncoder(sparse=False)
y_one_hot = onehot_encoder.fit_transform(y)


In [91]:
display(y_one_hot)

array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [92]:
classifier = LGBMClassifier(objective='multiclass', num_class=3, metric='multi_logloss', verbosity=-1)

# Definir el espacio de búsqueda de hiperparámetros
param_space = {
    'num_leaves': (10, 300),
    'learning_rate': (0.01, 0.5), 
    'n_estimators': (50, 300),
    'min_child_samples': (5, 100),
    'subsample': (0.5, 1.0),  
    'colsample_bytree': (0.5, 1.0)
}
# Definir función de pérdida para la búsqueda bayesiana
def multi_log_loss(estimator, X_test, y_test):
    y_pred = estimator.predict_proba(X_test)
    loss = log_loss(y_test, y_pred)
    return loss

# Definir validación cruzada temporal
cv = TimeSeriesSplit(n_splits=10).split(X)

# Realizar la búsqueda bayesiana de hiperparámetros
bayes_search = BayesSearchCV(
    classifier, 
    param_space, 
    scoring=multi_log_loss, 
    cv=cv,
    n_iter=50,
    verbose=0
)

# Realizar la búsqueda bayesiana
bayes_result = bayes_search.fit(X, y)

In [93]:
# Show best results
print("Best score:", bayes_result.best_score_)
print("Best parameters:", bayes_result.best_params_)

# Entrenar el modelo con los mejores hiperparámetros
best_model = bayes_result.best_estimator_
best_model.fit(X, y)

Best score: 3.3233412813590393
Best parameters: OrderedDict([('colsample_bytree', 1.0), ('learning_rate', 0.5), ('min_child_samples', 24), ('n_estimators', 300), ('num_leaves', 10), ('subsample', 0.5)])


LGBMClassifier(learning_rate=0.5, metric='multi_logloss', min_child_samples=24,
               n_estimators=300, num_class=3, num_leaves=10,
               objective='multiclass', subsample=0.5, verbosity=-1)

In [94]:
import json

# Obtener los hiperparámetros y puntajes de los 5 mejores modelos
top_n_models = 5
best_params_list = []
best_scores_list = []

for i in range(min(top_n_models, len(bayes_search.cv_results_['params']))):
    best_params_list.append(bayes_search.cv_results_['params'][i])
    best_scores_list.append(bayes_search.cv_results_['mean_test_score'][i])

# Guardar los hiperparámetros de los 5 mejores modelos en un archivo JSON
with open('gbm_classifier/top_5_hyperparameters.json', 'w') as f:
    json.dump({'best_params': best_params_list, 'best_scores': best_scores_list}, f)

# O imprimir los hiperparámetros
print("Top 5 mejores modelos:")
for i in range(len(best_params_list)):
    print("Modelo", i+1)
    print("Hiperparámetros:", best_params_list[i])
    print("Puntaje:", best_scores_list[i])


Top 5 mejores modelos:
Modelo 1
Hiperparámetros: OrderedDict([('colsample_bytree', 0.5885752220313429), ('learning_rate', 0.17082884484681185), ('min_child_samples', 6), ('n_estimators', 192), ('num_leaves', 231), ('subsample', 0.5085623038169307)])
Puntaje: 2.093090507673048
Modelo 2
Hiperparámetros: OrderedDict([('colsample_bytree', 0.7313732549225939), ('learning_rate', 0.47007092381868965), ('min_child_samples', 19), ('n_estimators', 291), ('num_leaves', 209), ('subsample', 0.9196262867394146)])
Puntaje: 2.847121139240301
Modelo 3
Hiperparámetros: OrderedDict([('colsample_bytree', 0.9392242353895535), ('learning_rate', 0.26428839588200526), ('min_child_samples', 13), ('n_estimators', 187), ('num_leaves', 32), ('subsample', 0.70993998152444)])
Puntaje: 2.541656866917237
Modelo 4
Hiperparámetros: OrderedDict([('colsample_bytree', 0.7804555919276301), ('learning_rate', 0.34732279373697456), ('min_child_samples', 54), ('n_estimators', 228), ('num_leaves', 145), ('subsample', 0.68558748

#### Armado del ensamble

In [95]:
# Crear una lista para almacenar los modelos individuales
individual_models = []

# Entrenar los modelos individuales
for params_str in best_params_list:
    model = LGBMClassifier(verbosity=-1, **params_str)
    model.fit(X, y)
    individual_models.append(model)

voting_model = VotingClassifier(estimators=[('model_'+str(i), model) for i, model in enumerate(individual_models)], voting='soft', verbose=0)
voting_model.fit(X, y)

with open('gbm_classifier/gbm_boosting_classifier.pkl', 'wb') as f:
    dill.dump(voting_model, f)

#### Clasificacion con el ensamble sobre las redicciones de los modelos generativos

In [96]:
with open('gbm_classifier/gbm_boosting_classifier.pkl', 'rb') as f:
    voting_model = dill.load(f)

In [97]:
voting_model.fit(X, y)

VotingClassifier(estimators=[('model_0',
                              LGBMClassifier(colsample_bytree=0.5885752220313429,
                                             learning_rate=0.17082884484681185,
                                             min_child_samples=6,
                                             n_estimators=192, num_leaves=231,
                                             subsample=0.5085623038169307,
                                             verbosity=-1)),
                             ('model_1',
                              LGBMClassifier(colsample_bytree=0.7313732549225939,
                                             learning_rate=0.47007092381868965,
                                             min_child_samples=19,
                                             n_estimators...
                              LGBMClassifier(colsample_bytree=0.7804555919276301,
                                             learning_rate=0.34732279373697456,
                                             min_child_samples=54,
                                             n_estimators=228, num_leaves=145,
                                             subsample=0.6855874898863495,
                                             verbosity=-1)),
                             ('model_4',
                              LGBMClassifier(colsample_bytree=0.7117112096391811,
                                             learning_rate=0.25324736023487676,
                                             min_child_samples=63,
                                             n_estimators=163, num_leaves=159,
                                             subsample=0.6187453051399433,
                                             verbosity=-1))],
                 verbose=0, voting='soft')

##### Datos originales:

In [98]:
display(clasifier_validation)

,Open_time,Open,High,Low,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,Volume_ETHUSDT,Number_of_trades_ETHUSDT,Close_BNBUSDT,Volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance,Tendencia
903,2024-03-18,10.08,10.46,9.60,245319.00,67609.99,55691.08,2464515.00,3520.46,570901.29,1906387.00,555.40,2284301.81,994512.00,10.06,9.95,11.86,10.06,8.26,52.48,34.00,43.00,84706.00,696,471,0.00,43.00,343.00,228.00,154000.00,Bajista
904,2024-03-19,9.90,9.99,8.60,341363.00,61937.40,101005.32,3593832.00,3158.64,1049629.69,2647385.00,507.70,2551361.51,1213572.00,10.08,9.84,11.81,10.08,8.35,42.93,120.00,126.00,135180.00,961,509,1.00,56.00,534.00,433.00,221000.00,Bajista
905,2024-03-20,8.77,9.57,8.49,267797.00,67840.51,90420.59,3549793.00,3516.53,1207322.82,2987953.00,556.80,1425296.58,809335.00,10.14,9.80,11.68,10.14,8.60,49.21,185.00,117.00,112997.00,866,555,1.00,40.00,473.00,386.00,171000.00,Alcista
906,2024-03-21,9.48,9.58,9.07,156774.00,65501.27,53357.48,2388390.00,3492.85,602755.21,1791989.00,553.80,953921.37,563996.00,10.17,9.74,11.63,10.17,8.71,46.85,64.00,81.00,66543.00,692,533,0.00,24.00,350.00,290.00,101000.00,Bajista
907,2024-03-22,9.18,9.37,8.69,147578.00,63796.64,51482.38,2492881.00,3336.35,558848.89,1747756.00,553.80,1181298.51,712381.00,10.14,9.67,11.67,10.14,8.62,45.00,57.00,66.00,68616.00,681,546,0.00,41.00,252.00,206.00,92000.00,Bajista


In [99]:
n_clases = 3 

validation_predictions = voting_model.predict(clasifier_validation.drop(columns=["Open_time", "Tendencia"]))
display(validation_predictions)

array(['Bajista', 'Bajista', 'Bajista', 'Alcista', 'Alcista'],
      dtype=object)

##### Datos generados por auto ml con prophet:

In [101]:
auto_ml_prophet_df = pd.read_csv('auto_timeseries_models_prophet/predicciones.csv')
auto_mp_prophet_predictions = voting_model.predict(auto_ml_prophet_df.drop(columns=["Open_time", "Close"]))
display(auto_mp_prophet_predictions)

array(['Bajista', 'Bajista', 'Bajista', 'Bajista', 'Bajista'],
      dtype=object)

##### Datos generados por auto ml con stats:

In [102]:
auto_ml_stats_df = pd.read_csv('auto_timeseries_models/predicciones.csv')
auto_mp_stats_predictions = voting_model.predict(auto_ml_stats_df.drop(columns=["Open_time", "Close"]))

display(auto_mp_stats_predictions)

array(['Bajista', 'Bajista', 'Bajista', 'Bajista', 'Bajista'],
      dtype=object)

##### Datos generados por auto ml con modelos clasicos:

In [103]:
auto_ml_df = pd.read_csv('h2o_models/predicciones.csv')
auto_mp_predictions = voting_model.predict(auto_ml_df.drop(columns=["Open_time", "Next_Day_Target", "Close"]))
display(auto_mp_predictions)

array(['Alcista', 'Alcista', 'Alcista', 'Alcista', 'Alcista'],
      dtype=object)